In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from utils import *

In [2]:
import tensorflow as tf
import keras

2024-01-26 21:51:44.155887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-26 21:51:44.155924: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
def optimise_conv(hp):
	channels = 6
	length = 400
	
	input_shape = (length, channels)
	num_classes = 1

	input_layer = keras.layers.Input(input_shape)

	conv1 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=256, step=32), kernel_size=hp.Int("kernel_size", min_value=2, max_value=12, step=2), padding="same")(input_layer)
	conv1 = keras.layers.BatchNormalization()(conv1)
	conv1 = keras.layers.ReLu()(conv1)

	conv2 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=256, step=32), kernel_size=hp.Int("kernel_size", min_value=2, max_value=12, step=2), padding="same")(conv1)
	conv2 = keras.layers.BatchNormalization()(conv2)
	conv2 = keras.layers.ReLu()(conv2)

	conv3 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=256, step=32), kernel_size=hp.Int("kernel_size", min_value=2, max_value=12, step=2), padding="same")(conv2)
	conv3 = keras.layers.BatchNormalization()(conv3)
	conv3 = keras.layers.ReLu()(conv3)

	gap = keras.layers.GlobalAveragePooling1D()(conv3)

	output_layer = keras.layers.Dense(num_classes, activation="sigmoid")(gap)

	model = keras.models.Model(inputs=input_layer, outputs=output_layer)

	model.compile(
		optimizer="adam",
		loss="binary_crossentropy",
		metrics=["accuracy"],
	)

	return model

In [ ]:
# dataset specific
channels = 6
data = 400

print("Training on dataset with ", data, "datapoints...")
(df, X_conv, y) = get_data_conv("train", data, channels)
(df, X_test_conv, y_test) = get_data_conv("test", data, channels)

In [ ]:
# train convolutional model
import keras_tuner

epochs = 60
batch_size = 32

print("Creating model...")
model = optimise_conv(keras_tuner.HyperParameters())

tuner = keras_tuner.RandomSearch(
    hypermodel=optimise_conv,
    objective="val_accuracy",
    executions_per_trial=2,
    overwrite=True,
    directory="./models/optimise/tuner",
    project_name="skiingai",
)

In [ ]:
print("Searcing for best configs model...")
tuner.search(X_conv, y, epochs=5, validation_data=(X_test_conv, y_test))

In [ ]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()

In [ ]:
result = best_model.evaluate(X_test_conv, y_test)
print("Accuracy of model", result[1])

In [ ]:
best_model.save("./models/optimised/best_tuned.keras")

In [ ]:
# Using best.keras convolutional model trained on 400 datapoints

model = tf.keras.models.load_model('./models/optimise/best.keras')
print("Evaluating model...")
result = model.evaluate(X_test_conv, y_test)
print("Accuracy of model for " + str(data) + " datapoints: ", result[1])

In [ ]:
# train optimised model in full

model = tf.keras.models.load_model('./models/optimise/best_tuned.keras')
model.compile(
	optimizer="adam",
	loss="binary_crossentropy",
	metrics=["accuracy"],
)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="./models/optimise/best_tuned_full.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
model.fit(X_conv, y, 
	batch_size=32,
	epochs=60,
	callbacks=[model_checkpoint_callback],
	validation_split=0.2,
	verbose=1
)

In [ ]:
model = tf.keras.models.load_model('./models/optimise/best_tuned.keras')
model.compile(
	optimizer="adam",
	loss="binary_crossentropy",
	metrics=["accuracy"],
)

result = model.evaluate(X_test_conv, y_test)
print("Accuracy of model for " + str(data) + " datapoints: ", result[1])